# Definição do Problema

## Objetivo

Por meio de redes neurais artificiais convolucionais (CNNs), busca-se realizar classificação multiclasse de imagens contendo diferentes animais.

## Conjunto de dados

O conjunto pode ser obtido [aqui](https://www.kaggle.com/datasets/alessiocorrado99/animals10).

Há dez tipos distintos de animais, sendo eles o cão, o cavalo, o elefante, a borboleta, a galinha, o gato, a vaca, a ovelha, a aranha e o esquilo, cada qual contendo suas respectivas variantes macho e fêmea. Trata-se de um conjunto com cerca de 26k exemplos, previamente rotulados, homogeneamente distribuído entre sete das dez classes, enquanto três delas apresentam quantia de instâncias significativamente superior às demais (vide gráfico na seção de "Leitura do conjunto de dados"). Ainda, em relação à composição desses dados, os exemplos se caracterizam, disjuntamente, como imagens conceituais, fotos, pinturas digitais e demais tipos visuais, embora a distribuição seja desconhecida para esse caso -- dada a falta de rotulação nesse sentido.

A saber, as imagens capturam os animais em diferentes ângulos, por vezes priorizando uma ou outra característica deles -- como, por exemplo, as asas para a borboleta, e as patas de um elefante, que visualmente se diferem em relação aos corpos dos demais animais. Além disso, algumas imagens contêm diversas ocorrências de um mesmo animal, como uma foto contendo uma ninhada de cães. Quanto aos planos de fundo, não há constância para as classes, de tal maneira que se torne seguro classificar tais animais sem ter de recorrer à análise do ambiente para isso; por exemplo, há fotos de esquilos em árvores e em solos coberto por folhas, mas também em ambientes urbanos e em telas planas.

Por fim, a divisão do conjunto de dados é realizada de forma pseudoaleatória, com semente fixa, por meio da técnica de Holdout. Desse modo, o referido conjunto é dividido nos subconjuntos de treino, de validação e de teste, cada qual com 80%, 10% e 10% da quantia absoluta de exemplos respectivamente. Para tanto, uma ferramenta utilitária do Keras será utilizada.

## Conceituação

Trata-se de um problema de classificação multiclasse por meio de aprendizado supervisionado. Ainda, por utilizar redes neurais artificiais, caracteriza-se como um paradigma conexionista, que será descrito por instâncias e aprenderá de forma incremental. Por conseguinte, não há descrição por meio de tabelas atributo-valor: ao invés disso, as características dos dados serão seus três canais (RGB), que são representados por matrizes de pixels, de modo aos modelos neurais inferirem novas características e realizarem o processo classificativo a partir de suas hipóteses.

## Plano de execução

Após a leitura e o pré-processamento dos dados, serão treinados cinco diferentes modelos neurais: um modelo denso, um modelo convolucional com poucas camadas, e as arquiteturas Xception, ResNet50 e InceptionV3. Para cada modelo, serão tomados os resultados que, ao final, serão comparados entre si. Ainda, será feita uma abordagem de visualização para compreensão do modo de extração de características por parte dos modelos convolucionais.


# Aquisição do conjunto de dados

Para execução em ambiente Google Colaboratory.

Para execução local, faça o download e o desempacotamento da base de dados no diretório deste notebook, 
no interior de uma pasta denominada "data". Caso não exista, crie uma. Ao final, é necessário que os 
dados estejam em ./data/raw-img/

In [ ]:
# Download do pacote do Kaggle
! pip install -q kaggle

In [ ]:
# Em sua conta do Kaggle, gere uma chave de API: isso fornecerá 
# um arquivo "kaggle.json". Faça upload dele nesta célula
from google.colab import files
files.upload()

In [ ]:
# Processamento da licença
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Importação do conjunto de dados
! kaggle datasets download 'alessiocorrado99/animals10'

In [ ]:
%%capture

# Unzip dos dados
! mkdir data
! unzip animals10.zip -d data

# Dependências

In [ ]:
# Para ciência de dados
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Model, save_model, load_model
from sklearn.model_selection import train_test_split

# Manipulação de arquivos
import os

# Para mostrar gráficos e imagens
from matplotlib import pyplot as plt
from IPython.display import Image, display
plt.style.use("bmh")

In [ ]:
# Verifica se a GPU está sendo utilizada e aplica a expansão memorial
physical_devices = tf.config.experimental.list_physical_devices("GPU")
print("Quantia de GPUs disponíveis:", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Leitura do conjunto de dados

In [ ]:
# Tradução de nomes presente em "./data/translate.py"
translation_mapper = {
    "cane": "dog", 
    "cavallo": "horse", 
    "elefante": "elephant", 
    "farfalla": "butterfly", 
    "gallina": "chicken", 
    "gatto": "cat", 
    "mucca": "cow", 
    "pecora": "sheep", 
    "ragno": "spider",
    "scoiattolo": "squirrel"
}

# Para mapeamento das labels
label_mapper = dict([(name, idx) for idx, name in enumerate(translation_mapper)])
label_unmapper = dict([(idx, name) for idx, name in enumerate(translation_mapper.values())])

# Diretório das imagens
IMG_DIR_PATH = "./data/raw-img/"

In [ ]:
# Visualização da distribuição de classes
class_names = os.listdir(IMG_DIR_PATH)
class_sizes = []
for class_name in class_names:
    class_sizes.append(len(os.listdir(IMG_DIR_PATH + class_name)))
class_names = list(map(translation_mapper.get, class_names))
plt.figure(figsize=(10,6))
x = np.arange(len(class_names))
plt.bar(x, class_sizes)
plt.xticks(x, class_names)
plt.show()
print("Total de elementos: {}".format(np.sum(class_sizes)))

In [ ]:
# Para manipulação dos dados
SEED = 16
IMAGE_SIZE = (224, 224)
load_dataset = lambda subset, split: tf.keras.utils.image_dataset_from_directory (
    IMG_DIR_PATH,
    labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=IMAGE_SIZE,
    shuffle=True,
    seed=SEED,
    subset=subset, 
    validation_split=split, 
)
train_data = load_dataset("training", 0.2)
val_data = load_dataset("validation", 0.2)
train_size = len(train_data.file_paths)
test_size = int(len(val_data.file_paths)/2)
test_data = val_data.take(test_size)
val_data.skip(test_size)
val_size = test_size
print("\nUsing {}, {} and {} files for training, validation and test respectively.".format(train_size, val_size, test_size))

In [ ]:
# Exemplo de imagens (possivelmente distorcidas devido ao redimensionamento)
plt.figure(figsize=(10, 10))
for images, labels in train_data.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(label_unmapper[int(labels[i])])
        plt.axis("off")

# Treinamento dos modelos

In [ ]:
# Função de perda e otimizador a serem utilizados
from keras.losses import SparseCategoricalCrossentropy
from keras.optimizers import Adam, SGD

# Para mapeamento
DENSE = "d"
CONVOLUTIONAL = "c"
XCEPTION = "x"
RESNET50 = "r"
INCEPTIONV3 = "i"

# Para guardas os resultados
results = {}

# Caminhos para os modelos salvos
model_paths = {
    DENSE: "./models/dense_model.h5",
    CONVOLUTIONAL: "./models/conv_model.h5", 
    XCEPTION: "./models/xception_model.h5", 
    RESNET50: "./models/resnet50.h5", 
    INCEPTIONV3: "./models/inceptionv3_model.h5", 
}
model_backup_paths = {
    DENSE: "./models/backups/dense_model.h5",
    CONVOLUTIONAL: "./models/backups/conv_model.h5", 
    XCEPTION: "./models/backups/xception_model.h5", 
    RESNET50: "./models/backups/resnet50.h5", 
    INCEPTIONV3: "./models/backups/inceptionv3_model.h5", 
}

# Tensor de entrada
input_tensor = keras.layers.Input(shape=(*IMAGE_SIZE, 3))

# Quantia de classes
num_classes = len(label_mapper)

## Baseline: RNA densa

In [ ]:
# Camadas necessárias
from keras.layers import Input, Flatten, Dense

# Criação do modelo
def create_dense_model(input_tensor, num_labels, loss, optimizer, name="model"):

    # Entrada
    inputs = input_tensor

    # Camadas Densas
    x = Flatten()(inputs[:,:,0]) # Para não explodir a memória: uso do primeiro canal somente
    x = Dense(units=256, activation="relu")(x)
    x = Dense(units=128, activation="relu")(x)
    x = Dense(units=64, activation="relu")(x)
    outputs = Dense(units=num_labels, activation="softmax")(x)

    # Finalização
    model = Model(inputs=inputs, outputs=outputs, name=name)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return model

# Modelo Denso
dense_model = None
if not os.path.exists(model_paths[DENSE]):
    loss_function = SparseCategoricalCrossentropy()
    model_optimizer = Adam(learning_rate = 0.00025, clipnorm = 1.0)
    dense_model = create_dense_model(input_tensor, num_classes, loss_function, model_optimizer, name="dense_model")
else:
    dense_model = load_model(model_paths[DENSE])

# Sumarização
dense_model.summary()

In [ ]:
# Treino
epochs = 6
dense_model.fit (
    train_data, 
    validation_data = val_data, 
    epochs = epochs, 
)
save_model(dense_model, model_backup_paths[DENSE])

In [ ]:
# Teste
results[DENSE] = dense_model.evaluate(test_data)
print("loss = {:.2f}; accuracy = {:.2f}%".format(results[DENSE][0], results[DENSE][1] * 100.0))

In [ ]:
# Salvar modelo
save_model(dense_model, model_paths[DENSE])

## Modelo Convolucional

In [ ]:
# Camadas necessárias
from keras.layers import Input, Activation, Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Criação do modelo
def create_conv_model(input_tensor, num_labels, loss, optimizer, name="model"):

    # Entrada
    inputs = input_tensor

    # Primeira seção
    x = Conv2D(32, kernel_size=(3, 3), padding='same')(inputs)
    x = Activation('relu')(x)
    x = Conv2D(32,kernel_size=(3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # Segunda seção
    x = Conv2D(64, kernel_size=(3, 3), padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(64, kernel_size=(3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)

    # Seção final
    x = Flatten()(x)
    x = Dense(512)(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_labels)(x)
    outputs = Activation(tf.nn.softmax)(x)

    # Finalização
    model = Model(inputs=inputs, outputs=outputs, name=name)
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    return model

# Modelo convolucional
conv_model = None
if not os.path.exists(model_paths[CONVOLUTIONAL]):
    loss_function = SparseCategoricalCrossentropy()
    model_optimizer = Adam(learning_rate = 0.00025, clipnorm = 1.0)
    conv_model = create_conv_model(input_tensor, num_classes, loss_function, model_optimizer, name="conv_model")
else:
    conv_model = load_model(model_paths[CONVOLUTIONAL])

# Sumarização
conv_model.summary()

In [ ]:
# Treino
epochs = 12
conv_model.fit (
    train_data, 
    validation_data = val_data, 
    epochs = epochs, 
)
save_model(conv_model, model_backup_paths[CONVOLUTIONAL])

In [ ]:
# Teste
results[CONVOLUTIONAL] = conv_model.evaluate(test_data)
print("loss = {:.2f}; accuracy = {:.2f}%".format(results[CONVOLUTIONAL][0], results[CONVOLUTIONAL][1] * 100.0))

In [ ]:
# Salvar modelo
save_model(conv_model, model_paths[CONVOLUTIONAL])

## Modelos pré-definidos

In [ ]:
# Camadas necessárias para a transferência de aprendizado
from keras.layers import Dense, GlobalAveragePooling2D

# Modelos originais
from keras.applications.xception import Xception
from keras.applications.resnet import ResNet50
from keras.applications.inception_v3 import InceptionV3

# Pré-processamento
from keras.applications.xception import preprocess_input as xception_preprocess_input
from keras.applications.resnet import preprocess_input as resnet_preprocess_input
from keras.applications.inception_v3 import preprocess_input as inception_preprocess_input

### Xception

In [ ]:
# Mapeamento dos conjuntos
def xception_preprocess_dataset(images, labels): 
    return xception_preprocess_input(images), labels
xception_train_data = train_data.map(xception_preprocess_dataset)
xception_val_data = val_data.map(xception_preprocess_dataset)
xception_test_data = test_data.map(xception_preprocess_dataset)

#### Nova arquitetura Xception

In [ ]:
# Modelo original
base_model = Xception(input_tensor=input_tensor, weights='imagenet', include_top=False)

# Adição de camadas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(num_classes, activation='softmax')(x)

# Novo modelo
xception_model = Model(inputs=base_model.input, outputs=outputs)

# Congelamento das camadas convolucionais (para treinar somente as últimas superiores)
for layer in base_model.layers:
    layer.trainable = False

# Compilação
loss_function = SparseCategoricalCrossentropy()
model_optimizer = Adam(learning_rate = 0.00025, clipnorm = 1.0)
xception_model.compile(optimizer=model_optimizer, loss=loss_function, metrics=['accuracy'])

In [ ]:
# Treino das camadas superiores
epochs = 8
xception_model.fit (
    xception_train_data, 
    validation_data = xception_val_data, 
    epochs = epochs, 
)
save_model(xception_model, model_backup_paths[XCEPTION])

In [ ]:
# Visualização das camadas (para escolher até onde descongelar)
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [ ]:
# Descongelamento das orimeiras camadas
for layer in xception_model.layers[:105]:
   layer.trainable = False
for layer in xception_model.layers[105:]:
   layer.trainable = True

# Recompilação
loss_function = SparseCategoricalCrossentropy()
model_optimizer = SGD(learning_rate=0.0001, momentum=0.9)
xception_model.compile(optimizer=model_optimizer, loss=loss_function, metrics=['accuracy'])

# Treino
epochs = 24
xception_model.fit (
    xception_train_data, 
    validation_data = xception_val_data, 
    epochs = epochs, 
)
save_model(xception_model, model_backup_paths[XCEPTION])

#### Carregamento de um modelo Xception

In [ ]:
# Carregamento do modelo
xception_model = load_model(model_paths[XCEPTION])

In [ ]:
# Treino
epochs = 1
xception_model.fit (
    xception_train_data, 
    validation_data = xception_val_data, 
    epochs = epochs, 
)
save_model(xception_model, model_backup_paths[XCEPTION])

#### Teste da Xception

In [ ]:
# Teste
results[XCEPTION] = xception_model.evaluate(xception_test_data)
print("loss = {:.2f}; accuracy = {:.2f}%".format(results[XCEPTION][0], results[XCEPTION][1] * 100.0))

In [ ]:
# Salvar modelo
save_model(xception_model, model_paths[XCEPTION])

### ResNet50

In [ ]:
# Mapeamento dos conjuntos
def resnet_preprocess_dataset(images, labels): 
    return resnet_preprocess_input(images), labels
resnet_train_data = train_data.map(resnet_preprocess_dataset)
resnet_val_data = val_data.map(resnet_preprocess_dataset)
resnet_test_data = test_data.map(resnet_preprocess_dataset)

#### Nova arquitetura ResNet50

In [ ]:
# Modelo original
base_model = ResNet50(input_tensor=input_tensor, weights='imagenet', include_top=False)

# Adição de camadas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(num_classes, activation='softmax')(x)

# Novo modelo
resnet_model = Model(inputs=base_model.input, outputs=outputs)

# Congelamento das camadas convolucionais (para treinar somente as últimas camadas)
for layer in base_model.layers:
    layer.trainable = False

# Compilação
loss_function = SparseCategoricalCrossentropy()
model_optimizer = Adam(learning_rate = 0.00025, clipnorm = 1.0)
resnet_model.compile(optimizer=model_optimizer, loss=loss_function, metrics=['accuracy'])

In [ ]:
# Treino das camadas superiores
epochs = 8
resnet_model.fit (
    resnet_train_data, 
    validation_data = resnet_val_data, 
    epochs = epochs, 
)
save_model(resnet_model, model_backup_paths[RESNET50])

In [ ]:
# Visualização das camadas (para escolher até onde descongelar)
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [ ]:
# Descongelamento das orimeiras camadas
for layer in resnet_model.layers[:154]:
   layer.trainable = False
for layer in resnet_model.layers[154:]:
   layer.trainable = True

# Recompilação
loss_function = SparseCategoricalCrossentropy()
model_optimizer = SGD(learning_rate=0.0001, momentum=0.9)
resnet_model.compile(optimizer=model_optimizer, loss=loss_function, metrics=['accuracy'])

# Treino
epochs = 32
resnet_model.fit (
    resnet_train_data, 
    validation_data = resnet_val_data, 
    epochs = epochs, 
)
save_model(resnet_model, model_backup_paths[RESNET50])

#### Carregamento de um modelo ResNet50

In [ ]:
# Carregamento do modelo
resnet_model = load_model(model_paths[RESNET50])

In [ ]:
# Treino
epochs = 1
resnet_model.fit (
    resnet_train_data, 
    validation_data = resnet_val_data, 
    epochs = epochs, 
)
save_model(resnet_model, model_backup_paths[RESNET50])

#### Teste da ResNet50

In [ ]:
# Teste
results[RESNET50] = resnet_model.evaluate(test_data)
print("loss = {:.2f}; accuracy = {:.2f}%".format(results[RESNET50][0], results[RESNET50][1] * 100.0))

In [ ]:
# Salvar modelo
save_model(resnet_model, model_paths[RESNET50])

### InceptionV3

In [ ]:
# Mapeamento dos conjuntos
def inception_preprocess_dataset(images, labels): 
    return inception_preprocess_input(images), labels
inception_train_data = train_data.map(inception_preprocess_dataset)
inception_val_data = val_data.map(inception_preprocess_dataset)
inception_test_data = test_data.map(inception_preprocess_dataset)

#### Nova arquitetura InceptionV3

In [ ]:
# Modelo original
base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet', include_top=False)

# Adição de camadas
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(num_classes, activation='softmax')(x)

# Novo modelo
inception_model = Model(inputs=base_model.input, outputs=outputs)

# Congelamento das camadas convolucionais (para treinar somente as camadas superiores)
for layer in base_model.layers:
    layer.trainable = False

# Compilação
loss_function = SparseCategoricalCrossentropy()
model_optimizer = Adam(learning_rate = 0.00025, clipnorm = 1.0)
inception_model.compile(optimizer=model_optimizer, loss=loss_function, metrics=['accuracy'])

In [ ]:
# Treino das camadas superiores
epochs = 8
inception_model.fit (
    inception_train_data, 
    validation_data = inception_val_data, 
    epochs = epochs, 
)
save_model(inception_model, model_backup_paths[INCEPTIONV3])

In [ ]:
# Visualização das camadas (para escolher até onde descongelar)
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [ ]:
# Descongelamento das orimeiras camadas
for layer in inception_model.layers[:249]:
   layer.trainable = False
for layer in inception_model.layers[249:]:
   layer.trainable = True

# Recompilação
loss_function = SparseCategoricalCrossentropy()
model_optimizer = SGD(learning_rate=0.0001, momentum=0.9)
inception_model.compile(optimizer=model_optimizer, loss=loss_function, metrics=['accuracy'])

# Treino
epochs = 32
inception_model.fit (
    inception_train_data, 
    validation_data = inception_val_data, 
    epochs = epochs, 
)
save_model(inception_model, model_backup_paths[INCEPTIONV3])

#### Carregamento de um modelo InceptionV3

In [ ]:
# Carregamento do modelo
inception_model = load_model(model_paths[INCEPTIONV3])

In [ ]:
# Treino
epochs = 8
inception_model.fit (
    inception_train_data, 
    validation_data = inception_val_data, 
    epochs = epochs, 
)
save_model(inception_model, model_backup_paths[INCEPTIONV3])

#### Teste da InceptionV3

In [ ]:
# Teste
results[INCEPTIONV3] = inception_model.evaluate(inception_test_data)
print("loss = {:.2f}; accuracy = {:.2f}%".format(results[INCEPTIONV3][0], results[INCEPTIONV3][1] * 100.0))

In [ ]:
# Salvar modelo
save_model(inception_model, model_paths[INCEPTIONV3])

# Resultados

In [ ]:
# Preparação
plt.figure(figsize=(10,6))
numeric_results = np.array(list(results.values()))
labels = ["Dense", "Convolutional", "Xception", "ResNet50", "InceptionV3"]
x = np.arange(numeric_results.shape[0])

# Barras
plt.bar(x-0.2, numeric_results[:,0], 0.4, label = 'Loss')
plt.bar(x+0.2, numeric_results[:,1], 0.4, label = 'Accuracy')

# Eixos
plt.xticks(x, labels)
plt.xlabel("NN Architecture")
plt.ylabel("Results")
plt.title("Metric results for different NN architectures")
plt.legend()

# Display
plt.show()

---

ESCREVER SOBRE OS RESULTADOS AQUI

# Visualização
Requer o treinamento do modelo InceptionV3

In [ ]:
# Dependências
from keras.applications.inception_v3 import decode_predictions, preprocess_input

# Imagem a ser utilizada
IMAGE_PATH = "./data/raw-img/elefante/e83cb60828f5043ed1584d05fb1d4e9fe777ead218ac104497f5c978a4eebdbd_640.jpg"

# Modelo auxiliar para visualizar as decisões
base_model = InceptionV3(input_tensor=input_tensor, weights='imagenet')
view_model = Model(inputs=base_model.inputs, outputs=base_model.layers[1].output)

# Imagem a ser utilizada
image = tf.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=IMAGE_SIZE)
display(image)
image = tf.keras.preprocessing.image.img_to_array(image)
image = preprocess_input(image)
image = np.expand_dims(image, axis=0)
preds = base_model.predict(image)
print(decode_predictions(preds, top=3)[0])

In [ ]:
# Extração de características e visualização
features = view_model.predict(image)
fig = plt.figure(figsize=(20,15))
for i in range(1,features.shape[3]+1):
    plt.subplot(8,8,i)
    plt.axis("off")
    plt.imshow(features[0,:,:,i-1] , cmap='gray')
plt.show()

In [ ]:
# Extração das camadas convolucionais e construção do modelo
conv_layers = []
for i, layer in enumerate(base_model.layers):
    if "conv" in layer.name:
        conv_layers.append(i)
head = conv_layers[:8] # Primeiras 8 camadas
tail = conv_layers[-8:] # Últimas 8 camadas
conv_layers = head + tail
outputs = [inception_model.layers[i].output for i in conv_layers]
view_model = Model(inputs=inception_model.inputs, outputs=outputs)

# Visualização
feature_map = view_model.predict(image)
for i, fmap in zip(conv_layers, feature_map):
    fig = plt.figure(figsize=(20,15))
    fig.suptitle("BLOCK_{}".format(i) , fontsize=20)
    for i in range(1,features.shape[3]+1):
        plt.subplot(8,8,i)
        plt.imshow(fmap[0,:,:,i-1] , cmap='gray')   
plt.show()